In [0]:
import numpy as np
import pandas as pd
import re
import collections

In [0]:
from nltk import word_tokenize as wt
from nltk import pos_tag as pt
from nltk.corpus import stopwords
from stanfordcorenlp import StanfordCoreNLP

In [0]:
nlp = StanfordCoreNLP(r'C:\Users\Kashyap\stanford-corenlp-full-2017-06-09')

In [0]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import GRU
from keras.layers import MaxoutDense
from keras.layers import Activation
from keras.utils import to_categorical

In [0]:
data1 = pd.read_csv("Project-2_Test_Data/Data-2_test.csv", sep='\s*,\s*')

In [0]:
data1['text'] = data1['text'].apply(lambda x: x.replace("[comma]", ',').replace("_", " ").strip())

data1['aspect_term'] = data1['aspect_term'].apply(lambda x: x.replace("[comma]", ',').replace("_", " ").strip())

In [0]:
# removing special chars
data1['text'] = data1['text'].apply(lambda x: re.sub('\W+',' ', x))

data1['aspect_term'] = data1['aspect_term'].apply(lambda x: re.sub('\W+',' ', x))

In [0]:
init_pos = []
for sent in data1['text']:
    init_pos.append(nlp.pos_tag(sent))

In [0]:
sw_set = set(stopwords.words('english'))

In [0]:
data1['text'] = data1['text'].apply(remove_stopwords)

data1['aspect_term'] = data1['aspect_term'].apply(remove_stopwords)

In [0]:
pos_mat = []
for sent in init_pos:
    pos_mat.append([x for x in sent if re.match('\W+', x[0]) is None and x[0] not in sw_set])

In [0]:
# 2nd method - more words in the window
feat_mat = []
pos_feat_mat = []
word_mat = []
progress = 0
prog = 0
for row,aspect in zip(pos_mat,data1['aspect_term']):
    prog += 1
    skip_row = False
    left_dist = []
    right_dist = []
    left_pos = []
    right_pos = []
    left_word = []
    right_word = []
    aspect_len = len(aspect.split())
    split_aspect = pt(wt(aspect))
    for i in range(len(row)):

        if row[i][0] == split_aspect[0][0]: 
        #or (split_aspect[0][0] in row[i][0])): 
        #and ((row[i+aspect_len-1][0] in split_aspect[aspect_len-1][0]) or (split_aspect[aspect_len-1][0] in row[i+aspect_len-1][0])):
            
            check = [row[q][0] for q in range(i,i+aspect_len)]
            #checking if all the words in aspect term occur together in the sentence (https://stackoverflow.com/a/9623147)
            if collections.Counter(check) == collections.Counter(aspect.split()):
                skip_row = True #Debug
                
                window = 10 - aspect_len
                left = window // 2 #number of words to traverse left
                right = window - left #number of words to traverse right
                
                l = i-left #index reached after moving left
                r = i+aspect_len-1+right #index reached after moving right
                
                
                if l < 0 and r > len(row)-1: #if there are not enough words on left and right
                    left += l
                    right -= r-len(row)+1
                
                elif l < 0: #if there are not enough words on left 
                    if r-l <= len(row)-1:
                        right += (-1)*l #take words from right
                    else:
                        right += len(row) - 1 - r
                    left -= (-1)*l
                        
                elif r > len(row)-1: #if there are not enough words on right 
                    if l-(r-len(row)+1) >= 0:
                        left += r-len(row)+1 #take words from left
                    else: 
                        left += l
                    right -= r-len(row)+1
                        
                #print(left) - Debug
                #print(i) - debug 
                #print(right) - debug
                #print(aspect) - debug
                
                count = 0
                j = 1
                    
                while count < left:
                    if row[i-j][0][0].isalnum():
                        count += 1
                        left_dist.append(-1*count)
                        left_pos.append(row[i-j][1])
                        left_word.append(row[i-j][0])
                    j += 1
                    
                count = 0
                #new_count = 0 
                #skips through all aspect terms
                j = aspect_len
            
                while count < right:
                    if row[i+j][0][0].isalnum() and row[i+j][0] not in split_aspect[0][0]:
                        count += 1
                        #if new:
                        #    right_dist.append(new_count)
                        #    new_count += 1
                        #else: 
                        right_dist.append(count)
                        right_pos.append(row[i+j][1])
                        right_word.append(row[i+j][0])

                    if row[i+j][0] in split_aspect[0][0]: #if first word of aspect and current word matches
                        if i+j+aspect_len-1 <= len(row)-1: 
                            check1 = [row[q][0] for q in range(i+j,i+j+aspect_len-1)]
                            if collections.Counter(check1) == collections.Counter(aspect.split()):
                                j += aspect_len-1
                            else:
                                count += 1
                                right_dist.append(count)
                                right_pos.append(row[i+j][1])
                                right_word.append(row[i+j][0])
                    
                        else: 
                            count += 1
                            right_dist.append(count)
                            right_pos.append(row[i+j][1])
                            right_word.append(row[i+j][0])
                                
                            #new = True
                            #right_dist.append(new_count)
                            #right_pos.append(row[i+j][1])
                            #right_word.append(row[i+j][0])
                            #new_count += 1
                            #count += 1    
                    j += 1
                progress += 1
                break
                
                
                
    pos_aspect = []
    for p in range(len(aspect.split())): 
        pos_aspect.append(row[i+p][1])
    
    if skip_row == False:
        print("{} row not considered in traversal".format(prog-1))
    
    rem = 10 - (aspect_len + left + right)
    
    if aspect_len <= 10:
        feat_mat.append([0]*rem + left_dist[::-1] + [0]*len(aspect.split()) + right_dist)
        pos_feat_mat.append(['NULL']*rem + left_pos[::-1] + pos_aspect + right_pos)
        word_mat.append([0]*rem + left_word[::-1] + aspect.split() + right_word)
        
    else: 
        feat_mat.append([0]*rem + left_dist[::-1] + [0]*10 + right_dist)
        pos_feat_mat.append(['NULL']*rem + left_pos[::-1] + pos_aspect[:10] + right_pos)
        word_mat.append([0]*rem + left_word[::-1] + aspect.split()[:10] + right_word)


In [0]:
#https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
#Tokenizing distances
t = Tokenizer()
t.fit_on_texts(feat_mat)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(feat_mat)

In [0]:
#Creating distance embeddings
model = Sequential()
e = Embedding(vocab_size, 50, input_length=10)
model.add(e)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
dist_embeddings = model.predict(np.array(encoded_docs))

In [0]:
# Pennington et. al., GloVe: Global Vectors for Word Representation
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.twitter.27B/glove.twitter.27B.50d.txt'
word2vec_output_file = 'glove.twitter.27B/glove.twitter.27B.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [0]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.twitter.27B/glove.twitter.27B.50d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [0]:
# Code taken from Peter Norvig's page http://norvig.com/spell-correct.html
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [0]:
# converting each word into its word embedding
zero_mat = np.zeros((50,))
glove_emb = []
for i,row in enumerate(word_mat): 
    row_vec = []
    for j,word in enumerate(row):
        if word == 0:
            row_vec.append(zero_mat)
        else:
            if word in model.vocab:
                vec = model.get_vector(word)
                row_vec.append(vec)
            elif correction(word) in model.vocab:
                vec = model.get_vector(correction(word))
                row_vec.append(vec)
            else:
                row_vec.append(zero_mat)
    glove_emb.append(np.array(row_vec))

glove_emb = np.array(glove_emb)

In [0]:
sum_embeddings = []
for i in range(progress):
    row = []
    for j in range(10):
        temp = dist_embeddings[i,j] + glove_emb[i,j]
        row.append(temp)
    sum_embeddings.append(np.array(row))

In [0]:
X = np.array(sum_embeddings)

In [0]:
Y = data1['class']
Y = Y.values

In [11]:
# Deep learning model 
model = Sequential()
model.add(GRU(100, return_sequences=False, input_shape=(X.shape[1],X.shape[2])))
model.add(MaxoutDense(100))
model.add(MaxoutDense(3))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:534: UserWarning: The `MaxoutDense` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `MaxoutDense` layer is deprecated '


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [13]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

skf = KFold(n_splits=10)
skf.get_n_splits(X, Y)
acc = []
pre = []
rec = []
f1 = []
for train_index, test_index in skf.split(X, Y):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    model.fit(X_train,y_train,epochs=14)
    y_pred = model.predict(X_test)
    acc.append(accuracy_score(y_test, y_pred.round()))
    f1.append(f1_score(y_test, y_pred.round(), average=None))
    pre.append(precision_score(y_test, y_pred.round(),average=None))
    rec.append(recall_score(y_test, y_pred.round(),average=None))
    
print("Average accuracy", sum(acc)/len(acc))
print("Average precision", np.mean(pre, axis=0))
print("Average recall", np.mean(rec, axis=0))
print("Average f1", np.mean(f1, axis=0))

Epoch 1/14
3241/3241 [==============================] - 4s 1ms/step - loss: 0.5040 - acc: 0.7621
Epoch 2/14
3241/3241 [==============================] - 3s 852us/step - loss: 0.4503 - acc: 0.7917
Epoch 3/14
3241/3241 [==============================] - 3s 860us/step - loss: 0.4306 - acc: 0.8071
Epoch 4/14
3241/3241 [==============================] - 3s 848us/step - loss: 0.4108 - acc: 0.8164
Epoch 5/14
3241/3241 [==============================] - 3s 846us/step - loss: 0.3911 - acc: 0.8272
Epoch 6/14
3200/3241 [============================>.] - ETA: 0s - loss: 0.3686 - acc: 0.8382

3241/3241 [==============================] - 3s 837us/step - loss: 0.3687 - acc: 0.8381
Epoch 7/14
3241/3241 [==============================] - 3s 839us/step - loss: 0.3453 - acc: 0.8477
Epoch 8/14
3241/3241 [==============================] - 3s 847us/step - loss: 0.3254 - acc: 0.8604
Epoch 9/14
3241/3241 [==============================] - 3s 858us/step - loss: 0.3027 - acc: 0.8698
Epoch 10/14
3241/3241 [==============================] - 3s 856us/step - loss: 0.2821 - acc: 0.8800
Epoch 11/14
3241/3241 [==============================] - 3s 847us/step - loss: 0.2621 - acc: 0.8874
Epoch 12/14
 224/3241 [=>............................] - ETA: 2s - loss: 0.2413 - acc: 0.8884

3241/3241 [==============================] - 3s 848us/step - loss: 0.2463 - acc: 0.8927
Epoch 13/14
3241/3241 [==============================] - 3s 847us/step - loss: 0.2260 - acc: 0.9018
Epoch 14/14
3241/3241 [==============================] - 3s 852us/step - loss: 0.2073 - acc: 0.9121
Epoch 1/14
3241/3241 [==============================] - 3s 852us/step - loss: 0.2394 - acc: 0.8993
Epoch 2/14
3241/3241 [==============================] - 3s 833us/step - loss: 0.2129 - acc: 0.9113
Epoch 3/14
3241/3241 [==============================] - 3s 862us/step - loss: 0.1976 - acc: 0.9171
Epoch 4/14
 224/3241 [=>............................] - ETA: 2s - loss: 0.1723 - acc: 0.9226

3241/3241 [==============================] - 3s 848us/step - loss: 0.1863 - acc: 0.9191
Epoch 5/14
3241/3241 [==============================] - 3s 840us/step - loss: 0.1717 - acc: 0.9264
Epoch 6/14
3241/3241 [==============================] - 3s 842us/step - loss: 0.1669 - acc: 0.9286
Epoch 7/14
3241/3241 [==============================] - 3s 840us/step - loss: 0.1531 - acc: 0.9335
Epoch 8/14
3241/3241 [==============================] - 3s 843us/step - loss: 0.1513 - acc: 0.9360
Epoch 9/14
3241/3241 [==============================] - 3s 837us/step - loss: 0.1480 - acc: 0.9351
Epoch 10/14
 224/3241 [=>............................] - ETA: 2s - loss: 0.1373 - acc: 0.9375

3241/3241 [==============================] - 3s 845us/step - loss: 0.1431 - acc: 0.9370
Epoch 11/14
3241/3241 [==============================] - 3s 850us/step - loss: 0.1389 - acc: 0.9392
Epoch 12/14
3241/3241 [==============================] - 3s 833us/step - loss: 0.1304 - acc: 0.9402
Epoch 13/14
3241/3241 [==============================] - 3s 848us/step - loss: 0.1305 - acc: 0.9430
Epoch 14/14
3241/3241 [==============================] - 3s 837us/step - loss: 0.1253 - acc: 0.9433
Epoch 1/14
3242/3242 [==============================] - 3s 825us/step - loss: 0.1430 - acc: 0.9374
Epoch 2/14
 512/3242 [===>..........................] - ETA: 2s - loss: 0.0972 - acc: 0.9603

...

3242/3242 [==============================] - 3s 847us/step - loss: 0.0274 - acc: 0.9883
Epoch 1/14
3242/3242 [==============================] - 3s 841us/step - loss: 0.0284 - acc: 0.9874
Epoch 2/14
3242/3242 [==============================] - 3s 846us/step - loss: 0.0274 - acc: 0.9888
Epoch 3/14
3242/3242 [==============================] - 3s 854us/step - loss: 0.0243 - acc: 0.9889
Epoch 4/14
3242/3242 [==============================] - 3s 843us/step - loss: 0.0279 - acc: 0.9880
Epoch 5/14
3242/3242 [==============================] - 3s 840us/step - loss: 0.0252 - acc: 0.9892
Epoch 6/14
 480/3242 [===>..........................] - ETA: 2s - loss: 0.0202 - acc: 0.9910

3242/3242 [==============================] - 3s 838us/step - loss: 0.0259 - acc: 0.9893
Epoch 7/14
3242/3242 [==============================] - 3s 850us/step - loss: 0.0245 - acc: 0.9894
Epoch 8/14
3242/3242 [==============================] - 3s 844us/step - loss: 0.0241 - acc: 0.9900
Epoch 9/14
3242/3242 [==============================] - 3s 856us/step - loss: 0.0260 - acc: 0.9893
Epoch 10/14
3242/3242 [==============================] - 3s 848us/step - loss: 0.0234 - acc: 0.9895
Epoch 11/14
3242/3242 [==============================] - 3s 866us/step - loss: 0.0237 - acc: 0.9907
Epoch 12/14
  32/3242 [..............................] - ETA: 2s - loss: 0.0076 - acc: 1.0000

3242/3242 [==============================] - 3s 856us/step - loss: 0.0213 - acc: 0.9911
Epoch 13/14
3242/3242 [==============================] - 3s 852us/step - loss: 0.0276 - acc: 0.9903
Epoch 14/14
3242/3242 [==============================] - 3s 855us/step - loss: 0.0185 - acc: 0.9919
Epoch 1/14
3242/3242 [==============================] - 3s 853us/step - loss: 0.0185 - acc: 0.9927
Epoch 2/14
3242/3242 [==============================] - 3s 852us/step - loss: 0.0213 - acc: 0.9917
Epoch 3/14
3242/3242 [==============================] - 3s 860us/step - loss: 0.0172 - acc: 0.9934


Epoch 4/14
3242/3242 [==============================] - 3s 856us/step - loss: 0.0229 - acc: 0.9918
Epoch 5/14
3242/3242 [==============================] - 3s 862us/step - loss: 0.0181 - acc: 0.9926
Epoch 6/14
3242/3242 [==============================] - 3s 864us/step - loss: 0.0173 - acc: 0.9933
Epoch 7/14
3242/3242 [==============================] - 3s 850us/step - loss: 0.0183 - acc: 0.9932
Epoch 8/14
3242/3242 [==============================] - 3s 860us/step - loss: 0.0190 - acc: 0.9930
Epoch 9/14
2912/3242 [=========================>....] - ETA: 0s - loss: 0.0156 - acc: 0.9939

3242/3242 [==============================] - 3s 847us/step - loss: 0.0167 - acc: 0.9938
Epoch 10/14
3242/3242 [==============================] - 3s 853us/step - loss: 0.0189 - acc: 0.9934
Epoch 11/14
3242/3242 [==============================] - 3s 866us/step - loss: 0.0155 - acc: 0.9949
Epoch 12/14
3242/3242 [==============================] - 3s 861us/step - loss: 0.0181 - acc: 0.9931
Epoch 13/14
3242/3242 [==============================] - 3s 851us/step - loss: 0.0160 - acc: 0.9935
Epoch 14/14
3242/3242 [==============================] - 3s 871us/step - loss: 0.0178 - acc: 0.9935
Average accuracy 0.9087650046168051
Average precision [0.89936677 0.82702683 0.9612767 ]
Average recall [0.8615797  0.85904924 0.93722894]
Average f1 [0.8774303  0.83680463 0.94789151]


In [0]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 100)               45300     
_________________________________________________________________
maxout_dense_3 (MaxoutDense) (None, 100)               40400     
_________________________________________________________________
maxout_dense_4 (MaxoutDense) (None, 3)                 1212      
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 86,912
Trainable params: 86,912
Non-trainable params: 0
_________________________________________________________________
None
